# Azure AI Search with LangChain

This code demonstrates how to use Azure AI Search with OpenAI and Langchain.

### Set up a Python virtual environment in Visual Studio Code

1. Open the Command Palette (Ctrl+Shift+P).
1. Search for **Python: Create Environment**.
1. Select **Venv**.
1. Select a Python interpreter. Choose 3.10 or later.

It can take a minute to set up. If you run into problems, see [Python environments in VS Code](https://code.visualstudio.com/docs/python/environments).

### Install packages

If you get an OS permission error, add `--user` to the command line.

In [1]:
! pip install -r azure-search-vector-python-langchain-sample-requirements.txt --quiet

### Load .env file

Copy `/code/.env-sample` to an `.env` file in the sample folder, and update accordingly. The search service and Azure OpenAI resource and model must exist, but the search index is created and loaded during code execution. Provide a unique name for the index. Endpoints and API keys can be found in the Azure portal.

In [2]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
import os

load_dotenv(override=True) # take environment variables from .env.

# Variables not used here do not need to be updated in your .env file
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
key_credential = os.environ["AZURE_SEARCH_ADMIN_KEY"] if len(os.environ["AZURE_SEARCH_ADMIN_KEY"]) > 0 else None
index_name = os.environ["AZURE_SEARCH_INDEX"]
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.environ["AZURE_OPENAI_KEY"] if len(os.environ["AZURE_OPENAI_KEY"]) > 0 else None
azure_openai_embedding_deployment = os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"]
azure_openai_api_version = os.environ["AZURE_OPENAI_API_VERSION"]

credential = key_credential or DefaultAzureCredential()

### Create LangChain Azure OpenAI Embeddings

In [3]:
from langchain_openai import AzureOpenAIEmbeddings
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

openai_credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(openai_credential, "https://cognitiveservices.azure.com/.default")

# Use API key if provided, otherwise use RBAC authentication
embeddings = AzureOpenAIEmbeddings(
    azure_deployment=azure_openai_embedding_deployment,
    openai_api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key,
    azure_ad_token_provider=token_provider if not azure_openai_key else None
)   

### Create LangChain Vector Store

In [4]:
from langchain.vectorstores.azuresearch import AzureSearch

vector_store = AzureSearch(
    azure_search_endpoint=endpoint,
    azure_search_key=key_credential,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
    semantic_configuration_name="default"
)

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

directory = os.path.join("..", "..", "..", "data", "documents")
files = ["Benefit_Options.pdf", "Northwind_Health_Plus_Benefits_Details.pdf", "Northwind_Standard_Benefits_Details.pdf"]
total_chunks = 0
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

for file in files:
    loader = PyPDFLoader(os.path.join(directory, file))
    file_chunks = loader.load_and_split(splitter)
    results = vector_store.add_documents(documents=file_chunks)
    total_chunks += len(results)
    print(f"Indexed {file}")
print(f"Indexed {total_chunks} chunks")

Indexed Benefit_Options.pdf
Indexed Northwind_Health_Plus_Benefits_Details.pdf
Indexed Northwind_Standard_Benefits_Details.pdf
Indexed 636 chunks


## Perform a vector similarity search

In [9]:
# Perform a similarity search
docs = vector_store.similarity_search(
    "What is included in my Northwind Health Plus plan that is not in standard?",
    k=50,
    search_type="similarity",
)
docs = docs[:3]
for doc in docs:
    print("-" * 80)  
    print(f"Source: {doc.metadata['source']}")
    print(f"Chunk Content: {doc.page_content}")

--------------------------------------------------------------------------------
Source: ..\..\..\data\documents\Benefit_Options.pdf
Chunk Content: Both plans offer coverage for pre scription drugs. Northwind Health Plus offers a wider range of 
prescription drug coverage than Northwind Standard. Northwind Health Plus covers generic, brand -
name, and specialty drugs, while Northwind Standard only covers generic and brand -name drugs. 
Both plans offer coverage for vision and dental services. Northwind Health Plus offers coverage for vision 
exams, glasses, and contact lenses, as well as dental exams, cleanings, and fillings. Northwind Standard 
only offers coverage for vision exams and glas ses. 
Both plans offer coverage for medical services. Northwind Health Plus offers coverage for hospital stays, 
doctor visits, lab tests, and X -rays. Northwind Standard only offers coverage for doctor visits and lab 
tests. 
Northwind Health Plus is a compr ehensive plan that offers more coverage

## Perform a hybrid search

In [10]:
# Perform a hybrid search
docs = vector_store.similarity_search(
    query="What is included in my Northwind Health Plus plan that is not in standard?",
    k=50, 
    search_type="hybrid"
)
docs = docs[:3]
for doc in docs:
    print("-" * 80)  
    print(f"Source: {doc.metadata['source']}")
    print(f"Chunk Content: {doc.page_content}")

--------------------------------------------------------------------------------
Source: ..\..\..\data\documents\Northwind_Health_Plus_Benefits_Details.pdf
Chunk Content: includes the plan documents that you receive from Northwind Health, the Northwind Health 
Plus plan summary, and any additional contracts or documents t hat you may have received 
from Northwind Health.  
It is important to remember that any changes made to this plan must be in writing and 
signed by both you and Northwind Health. Additionally, if something in the plan is not 
included in the plan documents or  summary, then it does not apply to the plan.  
You should also be aware that the Northwind Health Plus plan may contain certain 
exceptions, exclusions, and limitations. It is important to familiarize yourself with the plan 
documents to make sure that you u nderstand what services are covered and which are not 
covered. If you have any questions, Northwind Health has customer service representatives 
who are a

## Perform a hybrid search with semantic reranking (powered by Bing)

In [14]:
# Perform a hybrid search with semantic reranking  
docs_and_scores = vector_store.semantic_hybrid_search_with_score(  
    query="What is included in my Northwind Health Plus plan that is not in standard?",  
    k=50,  
)
docs_and_scores = docs_and_scores[:3]
# Print the results  
for doc, score in docs_and_scores:  
    print("-" * 80)  
    answers = doc.metadata['answers']  
    if answers:  
        if answers.get('highlights'):  
            print(f"Semantic Answer: {answers['highlights']}")  
        else:  
            print(f"Semantic Answer: {answers['text']}")  
        print(f"Semantic Answer Score: {score}")  
    print("Content:", doc.page_content)  
    captions = doc.metadata['captions']
    print(f"Score: {score}") 
    if captions:  
        if captions.get('highlights'):  
            print(f"Caption: {captions['highlights']}")  
        else:  
            print(f"Caption: {captions['text']}")  
    else:  
        print("Caption not available")  


--------------------------------------------------------------------------------
Content: Northwind Standard, you can choose from a variety of in -network providers, including primary care 
physicians, specialists, hospitals, and pharmacies. This plan  does not offer coverage for emergency 
services, mental health and substance abuse coverage, or out -of-network services.
Comparison of Plans 
Both plans offer coverage for routine physicals, well -child visits, immunizations, and other preventive 
care servi ces. The plans also cover preventive care services such as mammograms, colonoscopies, and 
other cancer screenings. 
Northwind Health Plus offers more comprehensive coverage than Northwind Standard. This plan offers 
coverage for emergency services, both in -network and out -of-network, as well as mental health and 
substance abuse coverage. Northwind Standard does not offer coverage for emergency services, mental 
health and substance abuse coverage, or out -of-network services.
Sc